### circle-ventures-portfolio monthly momemtum strategy

In [1]:
# get category symbols
category = "circle-ventures-portfolio"
import requests
url = "https://api.coingecko.com/api/v3/coins/markets"
params = {
    "vs_currency": "usd",
    "category": category,
    "order": "market_cap_desc",
    "per_page": 300,
    "page": 1
}

response = requests.get(url, params=params)
data = response.json()

# Extract the symbols
defi_symbols = [coin['symbol'].upper() + '/USDT' for coin in data]
print("DeFi Symbols:", defi_symbols)

DeFi Symbols: ['SUI/USDT', 'NEAR/USDT', 'APT/USDT', 'SEI/USDT', 'KCS/USDT', 'MINA/USDT', 'ZRO/USDT', 'BIGTIME/USDT', 'SPEC/USDT', 'CFG/USDT', 'MPL/USDT', 'TRIBL/USDT', 'MAV/USDT', 'ZBCN/USDT', 'BREED/USDT', 'EVMOS/USDT', 'XY/USDT', 'JPYC/USDT', 'DEVT/USDT', 'RATIO/USDT', 'FTT/USDT', 'JPYC/USDT']


In [2]:
# Fetch aviable symbols from Bybit
import ccxt

exchange = ccxt.bybit()
exchange.load_markets()
all_symbols = exchange.symbols

# Filter to include only DeFi symbols available on the exchange
available_defi_symbols = [symbol for symbol in defi_symbols if symbol in all_symbols]
print("Available DeFi Symbols:", available_defi_symbols)

Available DeFi Symbols: ['SUI/USDT', 'NEAR/USDT', 'APT/USDT', 'SEI/USDT', 'KCS/USDT', 'MINA/USDT', 'ZRO/USDT', 'SPEC/USDT', 'FTT/USDT']


In [3]:
# Fetch ohlcv from Bybit
import numpy as np
import pandas as pd
import datetime as dt
import pytz
import plotly.express as px
import plotly.graph_objects as go

symbols = available_defi_symbols
timeframe = '1d'
limit = 252

# Create an empty list to store DataFrames for each symbol
data_frames = []

for symbol in symbols:
    try:
        ohlcv_data = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
        df = pd.DataFrame(ohlcv_data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['symbol'] = symbol

        # Convert timestamp to datetime format for readability
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

        data_frames.append(df)
    except Exception as e:
        print(f"Could not fetch data for {symbol}: {e}")

# Concatenate all DataFrames into a single DataFrame
all_data = pd.concat(data_frames, ignore_index=True)
all_data.head()

,timestamp,open,high,low,close,volume,symbol
0,2024-03-10,1.5843,1.6514,1.5150,1.5550,8036897.01,SUI/USDT
1,2024-03-11,1.5550,1.5927,1.4755,1.5721,14040580.82,SUI/USDT
2,2024-03-12,1.5721,1.6353,1.4232,1.5991,24576085.45,SUI/USDT
3,2024-03-13,1.5991,1.6964,1.5733,1.6210,18274402.22,SUI/USDT
4,2024-03-14,1.6210,1.6603,1.4713,1.5674,18876518.39,SUI/USDT


In [4]:
# Check min max date
print(all_data['timestamp'].min(), all_data['timestamp'].max())

2024-03-10 00:00:00 2024-11-16 00:00:00


In [5]:
# Filter dublicates and unnecesary timestamps
# Ensure that 'timestamp' is in datetime format
all_data['timestamp'] = pd.to_datetime(all_data['timestamp'])

# Find the latest date in the data
latest_date = all_data['timestamp'].max()

# Calculate the cutoff date for 252 days before the latest date
cutoff_date = latest_date - pd.Timedelta(days=252)

# Filter the data to include only rows within the last 252 days
filtered_data = all_data[all_data['timestamp'] >= cutoff_date]

# Check the new date range to confirm
print("Filtered data range:", filtered_data['timestamp'].min(), "-", filtered_data['timestamp'].max())

# Identify duplicate entries based on 'timestamp' and 'symbol'
duplicates = filtered_data[filtered_data.duplicated(subset=['timestamp', 'symbol'], keep=False)]
print("Duplicate entries:")
duplicates.info()

# Remove duplicate entries, keeping only the first occurrence
filtered_data = filtered_data.drop_duplicates(subset=['timestamp', 'symbol'], keep='first')


Filtered data range: 2024-03-10 00:00:00 - 2024-11-16 00:00:00
Duplicate entries:
<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   timestamp  0 non-null      datetime64[ns]
 1   open       0 non-null      float64       
 2   high       0 non-null      float64       
 3   low        0 non-null      float64       
 4   close      0 non-null      float64       
 5   volume     0 non-null      float64       
 6   symbol     0 non-null      object        
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 0.0+ bytes


In [6]:
# get closed price data
df = filtered_data[['timestamp', 'close', 'symbol']]
df.head()

,timestamp,close,symbol
0,2024-03-10,1.5550,SUI/USDT
1,2024-03-11,1.5721,SUI/USDT
2,2024-03-12,1.5991,SUI/USDT
3,2024-03-13,1.6210,SUI/USDT
4,2024-03-14,1.5674,SUI/USDT


In [7]:
# pivot by timestamp and symbol
df_pivoted = df.pivot(index='timestamp', columns='symbol', values='close')

print(df_pivoted.head())
print(df_pivoted.tail())

symbol      APT/USDT  FTT/USDT  KCS/USDT  MINA/USDT  NEAR/USDT  SEI/USDT  \
timestamp                                                                  
2024-03-10   12.7483    2.3011   14.7883     1.5222     5.9470    0.8672   
2024-03-11   13.4642    2.2640   14.9854     1.6251     6.6823    0.8966   
2024-03-12   13.3778    2.3720   14.9645     1.5671     8.0733    0.9014   
2024-03-13   13.1462    2.4135   15.6701     1.5223     7.7623    0.9003   
2024-03-14   14.2954    2.5417   15.1017     1.4404     8.8575    0.8645   

symbol      SPEC/USDT  SUI/USDT  ZRO/USDT  
timestamp                                  
2024-03-10        NaN    1.5550       NaN  
2024-03-11        NaN    1.5721       NaN  
2024-03-12        NaN    1.5991       NaN  
2024-03-13        NaN    1.6210       NaN  
2024-03-14        NaN    1.5674       NaN  
symbol      APT/USDT  FTT/USDT  KCS/USDT  MINA/USDT  NEAR/USDT  SEI/USDT  \
timestamp                                                                  
2024-11

In [8]:
# Calculate monthly returns
returns_df = df_pivoted.pct_change().resample("ME").agg(lambda x: (x+1).prod()-1)
# Convert the index of past_cum_return_df to timezone-aware UTC
returns_df.index = returns_df.index.tz_localize('UTC')
returns_df.tail(7)

symbol,APT/USDT,FTT/USDT,KCS/USDT,MINA/USDT,NEAR/USDT,SEI/USDT,SPEC/USDT,SUI/USDT,ZRO/USDT
timestamp,,,,,,,,,
2024-05-31 00:00:00+00:00,0.050551,0.180277,0.041995,0.073622,0.170824,-0.064736,1.051827,-0.105438,0.000000
2024-06-30 00:00:00+00:00,-0.226417,-0.106719,-0.001706,-0.366546,-0.268194,-0.328140,-0.518847,-0.198593,0.012842
2024-07-31 00:00:00+00:00,-0.093225,-0.049283,-0.047967,-0.101776,-0.058934,-0.070443,0.246266,-0.137439,0.279426
2024-08-31 00:00:00+00:00,0.012428,-0.050882,-0.129704,-0.108158,-0.192103,-0.101674,-0.179538,0.125689,-0.089189
2024-09-30 00:00:00+00:00,0.181101,0.418965,-0.028072,0.288057,0.310809,0.624605,0.285676,1.228711,0.127480
2024-10-31 00:00:00+00:00,0.185478,-0.047226,0.100339,-0.059720,-0.233064,-0.166162,0.220705,0.111130,-0.272150
2024-11-30 00:00:00+00:00,0.373969,0.163429,0.238102,0.293153,0.506845,0.405553,0.249090,0.934676,0.110907


In [9]:
# Create a line plot with Plotly
fig = px.line(returns_df, x=returns_df.index, y=returns_df.columns, title="Monthly returns")
# Adjust the layout for a wider plot
fig.update_layout(width=1200, height=600, xaxis_title="Timestamp", yaxis_title="Returns")
# Show the interactive plot
fig.show()

In [10]:
# rolling cumulative for 6 months return
past_cum_return_df = (returns_df + 1).rolling(6).apply(np.prod) - 1

past_cum_return_df.tail(7)

symbol,APT/USDT,FTT/USDT,KCS/USDT,MINA/USDT,NEAR/USDT,SEI/USDT,SPEC/USDT,SUI/USDT,ZRO/USDT
timestamp,,,,,,,,,
2024-05-31 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-30 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-07-31 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-08-31 00:00:00+00:00,-0.490716,-0.439051,-0.439462,-0.715609,-0.321893,-0.671933,0.009471,-0.487974,0.180280
2024-09-30 00:00:00+00:00,-0.550847,-0.041800,-0.295872,-0.549960,-0.275182,-0.439690,0.297853,-0.073029,0.330742
2024-10-31 00:00:00+00:00,0.044644,0.286188,-0.078271,-0.340171,-0.345120,-0.289192,0.584295,0.723815,-0.031420
2024-11-30 00:00:00+00:00,0.366244,0.267828,0.095202,-0.205251,-0.157172,0.068231,-0.035530,2.728108,0.076002


In [11]:
# Show cumulative return from past 6 months
fig = px.line(past_cum_return_df, x=past_cum_return_df.index, y=past_cum_return_df.columns, title="Rolling cumulative for 6 months return")
fig.update_layout(width=1200, height=600, xaxis_title="Timestamp", yaxis_title="Cumulative Returns")
fig.show()

In [12]:
# Define formation and measurement period
# Define the dates with timezone information
end_of_measurement_period = dt.datetime(2024, 9, 30, tzinfo=pytz.UTC)
formation_period = dt.datetime(2024, 10, 31, tzinfo=pytz.UTC)

end_of_measurement_period_return_df = past_cum_return_df.loc[end_of_measurement_period]
# Transpose the DataFrame so that tickers become rows and returns become a single column
end_of_measurement_period_return_df = end_of_measurement_period_return_df.T
end_of_measurement_period_return_df = end_of_measurement_period_return_df.reset_index()
# Rename columns for clarity
end_of_measurement_period_return_df.columns = ['Symbol', 'Return']
end_of_measurement_period_return_df.head()

,Symbol,Return
0,APT/USDT,-0.550847
1,FTT/USDT,-0.041800
2,KCS/USDT,-0.295872
3,MINA/USDT,-0.549960
4,NEAR/USDT,-0.275182


In [13]:
# highest momentum in the positive direction
end_of_measurement_period_return_df.loc[end_of_measurement_period_return_df.iloc[:,1].idxmax()]

Symbol    ZRO/USDT
Return    0.330742
Name: 8, dtype: object

In [14]:
# highest momentum in the negative direction
end_of_measurement_period_return_df.loc[end_of_measurement_period_return_df.iloc[:,1].idxmin()]

Symbol    APT/USDT
Return   -0.550847
Name: 0, dtype: object

In [15]:
# rank symbols with quantiles
end_of_measurement_period_return_df['rank'] = pd.qcut(end_of_measurement_period_return_df.iloc[:,1], 3, labels=False)
end_of_measurement_period_return_df.head(7)

,Symbol,Return,rank
0,APT/USDT,-0.550847,0
1,FTT/USDT,-0.041800,2
2,KCS/USDT,-0.295872,1
3,MINA/USDT,-0.549960,0
4,NEAR/USDT,-0.275182,1
5,SEI/USDT,-0.439690,0
6,SPEC/USDT,0.297853,2


In [16]:
# Create a bar plot to visualize ranks
fig = px.bar(
    end_of_measurement_period_return_df, 
    x='Symbol', 
    y='Return', 
    color='rank',  # Color by rank to visually distinguish groups
    title="Asset Returns and Ranks",
    labels={'rank': 'Rank'},
)

# Customize layout for readability
fig.update_layout(
    xaxis_title="Symbol",
    yaxis_title="Return",
    coloraxis_colorbar=dict(title="Rank"),
    width=1000,
    height=600
)

fig.show()


In [17]:
# select highest rank
long_stocks = end_of_measurement_period_return_df.loc[end_of_measurement_period_return_df["rank"]==2,"Symbol"].values
long_stocks

array(['FTT/USDT', 'SPEC/USDT', 'ZRO/USDT'], dtype=object)

In [18]:
# select lowest rank
short_stocks = end_of_measurement_period_return_df.loc[end_of_measurement_period_return_df["rank"]==0,"Symbol"].values
short_stocks

array(['APT/USDT', 'MINA/USDT', 'SEI/USDT'], dtype=object)

In [19]:
# long strategy return
from dateutil.relativedelta import relativedelta

long_return_df = returns_df.loc[formation_period + relativedelta(months=1), \
                                   returns_df.columns.isin(long_stocks)]
long_return_df

symbol
FTT/USDT     0.163429
SPEC/USDT    0.249090
ZRO/USDT     0.110907
Name: 2024-11-30 00:00:00+00:00, dtype: float64

In [20]:
# short strategy return
short_return_df = returns_df.loc[formation_period + relativedelta(months=1), \
                                   returns_df.columns.isin(short_stocks)]
short_return_df

symbol
APT/USDT     0.373969
MINA/USDT    0.293153
SEI/USDT     0.405553
Name: 2024-11-30 00:00:00+00:00, dtype: float64

In [25]:
# Define Strategy tester class
class strategyTester:
    """Calculate metrics based on cumulative return"""

    def __init__(self, df: pd.Series, period: str, riskfree_rate: float = 0.03, initial_wealth: int = 1000):
        self.annualized_return_value = self.annualized_return(df, period)
        self.annualized_volatility_value = self.annualized_volatility(df, period)
        self.sharpe_ratio_value = self.sharpe_ratio(df, riskfree_rate)
        self.max_drawdown_value = self.max_drawdown(df, initial_wealth)
        
    # annualized return
    @staticmethod
    def annualized_return(df: pd.Series, period: str):
        if period == 'daily':
            factor = 252
        elif period == 'weekly':
            factor = 52
        elif period == 'monthly':
            factor = 12
        else:
            raise ValueError("Invalid period. Use 'daily', 'weekly', or 'monthly'.")
        
        annualized_return = (1 + df) ** (factor / len(df)) - 1
        print(f"Annualized return: {annualized_return}")
        return annualized_return

    # annualized volatility
    @staticmethod
    def annualized_volatility(df: pd.Series, period: str):
        if period == 'daily':
            factor = 252
        elif period == 'weekly':
            factor = 52
        elif period == 'monthly':
            factor = 12
        else:
            raise ValueError("Invalid period. Use 'daily', 'weekly', or 'monthly'.")
        
        volatility = df.std() * (factor ** 0.5)
        print(f"Annualized volatility: {volatility}")
        return volatility

    # Sharpe ratio
    @staticmethod
    def sharpe_ratio(df: pd.Series, riskfree_rate: float = 0.03):
        excess_return = df.mean() - riskfree_rate
        volatility = df.std()
        sharpe_ratio = excess_return / volatility if volatility != 0 else np.nan
        print(f"Sharpe ratio: {sharpe_ratio}")
        return sharpe_ratio

    # Drawdown calculation
    @staticmethod
    def drawdown(return_series: pd.Series, initial_wealth: float = 100):
        prior_peaks_series = return_series.cummax()
        drawdown_series = (return_series - prior_peaks_series) / prior_peaks_series
        return pd.DataFrame({
            "Wealth Index": return_series,
            "Prior Peaks": prior_peaks_series,
            "Drawdown": drawdown_series
        })
    
    # Max drawdown
    @staticmethod
    def max_drawdown(df: pd.Series, initial_wealth):
        drawdown_df = strategyTester.drawdown(df, initial_wealth)
        max_drawdown = drawdown_df["Drawdown"].min()
        print(f"Max drawdown: {max_drawdown} with initial wealth: {initial_wealth}")
        return max_drawdown


In [26]:
# long momentum strategy test
long_momentum_strategy_test = strategyTester(long_return_df, 'monthly', 0.03, 1000)

Annualized return: symbol
FTT/USDT     0.832144
SPEC/USDT    1.434302
ZRO/USDT     0.523040
Name: 2024-11-30 00:00:00+00:00, dtype: float64
Annualized volatility: 0.24162204873675447
Sharpe ratio: 2.0713226726368315
Max drawdown: -0.5547492466365329 with initial wealth: 1000


In [27]:
# short momentum strategy test
short_momentum_strategy_test = strategyTester(short_return_df, 'monthly', 0.03, 1000)

Annualized return: symbol
APT/USDT     2.563751
MINA/USDT    1.796400
SEI/USDT     2.902910
Name: 2024-11-30 00:00:00+00:00, dtype: float64
Annualized volatility: 0.2008108535177171
Sharpe ratio: 5.650563028559416
Max drawdown: -0.2161034775899059 with initial wealth: 1000


In [ ]:
# Comparison of monthly Long and Short momentum strategies
# Initialize an empty figure
fig = go.Figure()

# Add the first strategy (e.g., long strategy)
fig.add_trace(go.Bar(x=long_return_df.index, y=long_return_df.values, marker_color='blue', name="Long"))

# Add the second strategy (e.g., short strategy)
fig.add_trace(go.Bar(x=short_return_df.index, y=short_return_df.values, marker_color='red', name="Short"))

# Update layout for title and axis labels
fig.update_layout(
    title="Comparison of monthly Long and Short momentum strategies",
    width=1200,
    height=400,
    xaxis_title="Symbol",
    yaxis_title="Cumulative Returns",
    legend_title="Strategies",
)
# Show the plot
fig.show()

In [28]:
# Annualized Return and Max Drawdown from Long and Short Strategies
# Initialize an empty figure
fig = go.Figure()

# Add traces for Long strategy
fig.add_trace(go.Bar(
    x=["Annualized Return (Long)", "Max Drawdown (Long)"], 
    y=[long_momentum_strategy_test.annualized_return_value.mean(), long_momentum_strategy_test.max_drawdown_value], 
    marker_color='blue', 
    name="Long"
))

# Add traces for Short strategy
fig.add_trace(go.Bar(
    x=["Annualized Return (Short)", "Max Drawdown (Short)"], 
    y=[short_momentum_strategy_test.annualized_return_value.mean(), short_momentum_strategy_test.max_drawdown_value], 
    marker_color='red', 
    name="Short"
))

# Update layout
fig.update_layout(
    title="Annualized Return and Max Drawdown from Long and Short Strategies",
    xaxis_title="Metrics",
    yaxis_title="Values",
    barmode='group',  # Group bars by category
    width=800,
    height=400
)

# Show the plot
fig.show()



In [29]:
# Annualized Volatility and Max Drawdown from Long and Short Strategies
# Create a figure
fig = go.Figure()

# Add bars for the Long Strategy - Volatility and Drawdown
fig.add_trace(go.Bar(
    x=["Annualized Volatility(long)", "Sharpe Ratio(long)"], 
    y=[long_momentum_strategy_test.annualized_volatility_value, long_momentum_strategy_test.sharpe_ratio_value],
    name="Long",
    marker_color='blue'
))

# Add bars for the Short Strategy - Volatility and Drawdown
fig.add_trace(go.Bar(
    x=["Annualized Volatility(short)", "Sharpe Ratio(short)"], 
    y=[short_momentum_strategy_test.annualized_volatility_value, short_momentum_strategy_test.sharpe_ratio_value],
    name="Short",
    marker_color='red'
))

# Update layout for grouped bars
fig.update_layout(
    title="Annualized Volatility and Max Drawdown from Long and Short Strategies",
    xaxis_title="Metrics",
    yaxis_title="Metric Value",
    barmode='group',  # Group bars side-by-side
    width=800,
    height=400
)

# Show the plot
fig.show()
